In [1]:
#### label 정보
#### (배경 - 0 / 몸통 - 1 / 오른손 - 2 / 왼손 - 3 / 왼발 - 4 / 오른발 - 5 / 오른쪽 허벅지 - 6 / 왼쪽 허벅지 - 7 / 오른쪽 종아리 - 8 / 왼쪽 종아리 - 9 / 왼쪽 팔 - 10 / 
#### 오른쪽 팔 - 11 / 왼쪽 전완 - 12 / 오른쪽 전완 - 13 / 머리 - 14)
#### 해당 값이 RGB 값임 ex) 배경 0,0,0
### mask 확장자 png input image 확장자 jpg 해깔리지 말것

In [2]:
# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Check MMSegmentation installation
import mmseg
print(mmseg.__version__)
import mmcv
import os.path as osp

1.9.0 True
0.18.0


In [3]:
from mmseg.apis import inference_segmentor, init_segmentor, show_result_pyplot
from mmseg.core.evaluation import get_palette

In [4]:
classes = ('bg', 'body', 'right_hand', 'left_hand', 'left_leg', 'right_reg', 'right_thigh', 'left_thigh','right_calf','left_calf'
           ,'left_arm','right_arm','left_forearm','right_forearm','head')

In [5]:
palette = [[0,0,0], [1,1,1], [2,2,2], [3,3,3],[4,4,4], [5,5,5], [6,6,6], [7,7,7], [8,8,8], [9,9,9], [10,10,10], [11,11,11], [12,12,12], [13,13,13], [14,14,14]]

In [6]:
from mmseg.datasets.builder import DATASETS
from mmseg.datasets.custom import CustomDataset

@DATASETS.register_module()
class body_seg(CustomDataset):
  CLASSES = classes
  PALETTE = palette
  def __init__(self, split, **kwargs):
    super().__init__(img_suffix='.jpg', seg_map_suffix='.png', 
                     split=split, **kwargs)
    assert osp.exists(self.img_dir) and self.split is not None


In [7]:
from mmcv import Config

In [8]:
from mmseg.apis import set_random_seed
cfg = Config.fromfile('mmsegmentation/configs/swin/upernet_swin_tiny_patch4_window7_512x512_160k_ade20k_pretrain_224x224_1K.py')

In [9]:
cfg.dataset_type = 'body_seg'
cfg.data_root = '/root'


# Since we use ony one GPU, BN is used instead of SyncBN
## encoding 부분은 layer normalization 
## encoding 이후 ecoder part 부터는 batchnormalization
cfg.norm_cfg = dict(type='LN', requires_grad=True)
cfg.model.backbone.norm_cfg = cfg.norm_cfg
cfg.model.decode_head.norm_cfg = dict(type='BN', requires_grad=True)
cfg.model.auxiliary_head.norm_cfg = dict(type='BN', requires_grad=True)


# modify num classes of the model in decode/auxiliary head
cfg.model.decode_head.num_classes = 15
cfg.model.auxiliary_head.num_classes = 15

cfg.data.samples_per_gpu = 24
cfg.data.workers_per_gpu = 2

cfg.img_norm_cfg = dict(
    mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_rgb=True)
cfg.crop_size = (224, 224)

cfg.train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations'),
    dict(type='Resize', img_scale=(256, 256), ratio_range=(0.5, 2.0)),
    dict(type='RandomCrop', crop_size=cfg.crop_size, cat_max_ratio=0.75),
    dict(type='RandomFlip', flip_ratio=0.5),
    dict(type='PhotoMetricDistortion'),
    dict(type='Normalize', **cfg.img_norm_cfg),
    dict(type='Pad', size=cfg.crop_size, pad_val=0, seg_pad_val=255),
    dict(type='DefaultFormatBundle'),
    dict(type='Collect', keys=['img', 'gt_semantic_seg']),
]

cfg.test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(
        type='MultiScaleFlipAug',
        img_scale=(224, 224),
        # img_ratios=[0.5, 0.75, 1.0, 1.25, 1.5, 1.75],
        flip=False,
        transforms=[
            dict(type='Resize', keep_ratio=True),
            dict(type='Normalize', **cfg.img_norm_cfg),
            dict(type='ImageToTensor', keys=['img']),
            dict(type='Collect', keys=['img']),
        ])
]

cfg.data.train.type = cfg.dataset_type
cfg.data.train.data_root = cfg.data_root
cfg.data.train.img_dir = 'train2014'
cfg.data.train.ann_dir = 'train_mask'
cfg.data.train.pipeline = cfg.train_pipeline
cfg.data.train.split = 'splits/train.txt'

cfg.data.val.type = cfg.dataset_type
cfg.data.val.data_root = cfg.data_root
cfg.data.val.img_dir = 'val2014'
cfg.data.val.ann_dir = 'val_mask'
cfg.data.val.pipeline = cfg.test_pipeline
cfg.data.val.split = 'splits/val.txt'


cfg.data.test.type = cfg.dataset_type
cfg.data.test.data_root = cfg.data_root
cfg.data.test.img_dir = 'val2014'
cfg.data.test.ann_dir = 'val_mask'
cfg.data.test.pipeline = cfg.test_pipeline
cfg.data.test.split = 'splits/val.txt'

checkpoint_file = 'pretrained/upernet_swin_tiny_patch4_window7_512x512_160k_ade20k_pretrain_224x224_1K.pth'
cfg.load_from = checkpoint_file

# Set up working dir to save files and logs.
cfg.work_dir = './work_dirs/train_tiny'

# 1epoch 대략 43000
cfg.runner.max_iters = 10833*50
cfg.log_config.interval = 3610
cfg.evaluation.interval = 10833
cfg.checkpoint_config.interval = 10833

# Set seed to facitate reproducing the result
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)

print(f'Config:\n{cfg.pretty_text}')

Config:
norm_cfg = dict(type='LN', requires_grad=True)
backbone_norm_cfg = dict(type='LN', requires_grad=True)
model = dict(
    type='EncoderDecoder',
    pretrained='pretrain/swin_tiny_patch4_window7_224.pth',
    backbone=dict(
        type='SwinTransformer',
        pretrain_img_size=224,
        embed_dims=96,
        patch_size=4,
        window_size=7,
        mlp_ratio=4,
        depths=[2, 2, 6, 2],
        num_heads=[3, 6, 12, 24],
        strides=(4, 2, 2, 2),
        out_indices=(0, 1, 2, 3),
        qkv_bias=True,
        qk_scale=None,
        patch_norm=True,
        drop_rate=0.0,
        attn_drop_rate=0.0,
        drop_path_rate=0.3,
        use_abs_pos_embed=False,
        act_cfg=dict(type='GELU'),
        norm_cfg=dict(type='LN', requires_grad=True)),
    decode_head=dict(
        type='UPerHead',
        in_channels=[96, 192, 384, 768],
        in_index=[0, 1, 2, 3],
        pool_scales=(1, 2, 3, 6),
        channels=512,
        dropout_ratio=0.1,
        num_cla

In [10]:
from mmseg.datasets import build_dataset
from mmseg.models import build_segmentor
from mmseg.apis import train_segmentor


# Build the dataset
datasets = [build_dataset(cfg.data.train)]

# Build the detector

# config_file = 'mmsegmentation/configs/swin/upernet_swin_base_patch4_window12_512x512_160k_ade20k_pretrain_384x384_22K.py'
# model = init_segmentor(config_file, checkpoint_file, device='cuda:0')
model = build_segmentor(
    cfg.model, train_cfg=cfg.get('train_cfg'), test_cfg=cfg.get('test_cfg'))

# Add an attribute for visualization convenience
model.CLASSES = datasets[0].CLASSES

# Create work_dir
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
train_segmentor(model, datasets, cfg, distributed=False, validate=True, 
                meta=dict())

/opt/conda/lib/python3.7/site-packages/mmcv/utils/misc.py:334: UserWarning: "flip_ratio" is deprecated in `RandomFlip.__init__`, please use "prob" instead
  f'"{src_arg_name}" is deprecated in '
2021-10-30 00:44:48,215 - mmseg - INFO - Loaded 26437 images
/opt/conda/lib/python3.7/site-packages/mmseg/models/backbones/swin.py:553: UserWarning: DeprecationWarning: pretrained is deprecated, please use "init_cfg" instead
  warnings.warn('DeprecationWarning: pretrained is deprecated, '
2021-10-30 00:45:03,515 - mmseg - INFO - Loaded 1508 images
2021-10-30 00:45:03,516 - mmseg - INFO - load checkpoint from pretrained/upernet_swin_tiny_patch4_window7_512x512_160k_ade20k_pretrain_224x224_1K.pth
2021-10-30 00:45:03,517 - mmseg - INFO - Use load_from_local loader
2021-10-30 00:45:04,931 - mmseg - WARNING - The model and loaded state dict do not match exactly

size mismatch for decode_head.conv_seg.weight: copying a param with shape torch.Size([150, 512, 1, 1]) from checkpoint, the shape in curren

[                                ] 2/1508, 5.9 task/s, elapsed: 0s, ETA:   257s

[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)


[>>>>>>>>>>>>>>>>>>>>>>>>>>>] 1508/1508, 23.7 task/s, elapsed: 64s, ETA:     0s

2021-10-30 02:47:58,556 - mmseg - INFO - per class results:
2021-10-30 02:47:58,558 - mmseg - INFO - 
+---------------+-------+-------+
|     Class     |  IoU  |  Acc  |
+---------------+-------+-------+
|       bg      | 91.93 |  97.5 |
|      body     |  55.9 | 72.26 |
|   right_hand  | 28.91 | 45.26 |
|   left_hand   |  5.39 |  6.07 |
|    left_leg   |  6.23 |  7.67 |
|   right_reg   |  8.9  | 11.12 |
|  right_thigh  | 26.25 | 45.12 |
|   left_thigh  |  3.78 |  4.15 |
|   right_calf  |  12.5 | 17.51 |
|   left_calf   | 13.17 | 19.54 |
|    left_arm   |  6.54 |  7.1  |
|   right_arm   |  26.0 | 40.31 |
|  left_forearm |  7.45 |  8.61 |
| right_forearm | 23.25 | 38.55 |
|      head     | 67.55 | 79.18 |
+---------------+-------+-------+
2021-10-30 02:47:58,558 - mmseg - INFO - Summary:
2021-10-30 02:47:58,559 - mmseg - INFO - 
+-------+-------+-------+
|  aAcc |  mIoU |  mAcc |
+-------+-------+-------+
| 89.71 | 25.58 | 33.33 |
+-------+-------+-------+
2021-10-30 02:47:58,561 - mmse

...

[>>>>>>>>>>>>>>>>>>>>>>>>>>>] 1508/1508, 23.8 task/s, elapsed: 63s, ETA:     0s

2021-11-01 01:47:14,318 - mmseg - INFO - per class results:
2021-11-01 01:47:14,320 - mmseg - INFO - 
+---------------+-------+-------+
|     Class     |  IoU  |  Acc  |
+---------------+-------+-------+
|       bg      | 93.13 | 97.56 |
|      body     | 61.38 | 74.76 |
|   right_hand  | 28.18 | 39.56 |
|   left_hand   | 20.86 | 28.92 |
|    left_leg   | 14.21 | 19.49 |
|   right_reg   | 14.65 | 20.24 |
|  right_thigh  | 22.01 | 31.44 |
|   left_thigh  |  21.9 | 34.16 |
|   right_calf  | 19.95 | 28.34 |
|   left_calf   | 19.35 | 28.56 |
|    left_arm   | 22.77 | 36.26 |
|   right_arm   | 22.08 | 31.09 |
|  left_forearm | 20.31 | 30.78 |
| right_forearm | 23.81 | 34.08 |
|      head     | 71.23 | 82.02 |
+---------------+-------+-------+
2021-11-01 01:47:14,321 - mmseg - INFO - Summary:
2021-11-01 01:47:14,321 - mmseg - INFO - 
+-------+-------+-------+
|  aAcc |  mIoU |  mAcc |
+-------+-------+-------+
| 90.65 | 31.72 | 41.15 |
+-------+-------+-------+
2021-11-01 01:47:14,324 - mmse

[>>>>>>>>>>>>>>>>>>>>>>>>>>>] 1508/1508, 23.8 task/s, elapsed: 63s, ETA:     0s

2021-11-01 03:48:57,925 - mmseg - INFO - per class results:
2021-11-01 03:48:57,926 - mmseg - INFO - 
+---------------+-------+-------+
|     Class     |  IoU  |  Acc  |
+---------------+-------+-------+
|       bg      | 93.04 | 97.59 |
|      body     | 60.97 | 74.16 |
|   right_hand  | 26.29 | 37.29 |
|   left_hand   | 19.61 | 28.27 |
|    left_leg   | 12.25 | 16.91 |
|   right_reg   | 13.25 | 18.63 |
|  right_thigh  | 21.53 | 31.63 |
|   left_thigh  | 19.48 | 29.64 |
|   right_calf  | 19.06 | 27.44 |
|   left_calf   | 18.06 | 26.16 |
|    left_arm   | 21.95 |  34.7 |
|   right_arm   | 21.83 | 31.04 |
|  left_forearm | 19.24 | 28.75 |
| right_forearm | 23.67 | 34.19 |
|      head     | 70.95 | 81.51 |
+---------------+-------+-------+
2021-11-01 03:48:57,927 - mmseg - INFO - Summary:
2021-11-01 03:48:57,927 - mmseg - INFO - 
+-------+-------+-------+
|  aAcc |  mIoU |  mAcc |
+-------+-------+-------+
| 90.52 | 30.74 | 39.86 |
+-------+-------+-------+
2021-11-01 03:48:57,930 - mmse

KeyboardInterrupt: 

In [ ]:
model